In [15]:
# neck
from keras.applications.inception_v3 import InceptionV3,preprocess_input
from keras.applications.vgg16 import VGG16,preprocess_input

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

HEIGHT = 224
WIDTH = 224
TRAIN_DIR = "train_category/Men's_wear/Attributes/sleeve"
BATCH_SIZE = 8

# create the base pre-trained model
#base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(HEIGHT, WIDTH, 3))

In [16]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        TRAIN_DIR,
        target_size=(HEIGHT, WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical')

Found 84 images belonging to 2 classes.


In [18]:
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model

def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model

class_list = ["full_sleeve", "short_sleeve"]
FC_LAYERS = [128, 128]
dropout = 0.5

finetune_model = build_finetune_model(base_model, 
                                      dropout=dropout, 
                                      fc_layers=FC_LAYERS, 
                                      num_classes=len(class_list))


In [19]:
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint

NUM_EPOCHS = 30
BATCH_SIZE = 8


adam = Adam(lr=0.001)
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

filepath="checkpoints\\" + "VGG16" + "sleeve_model_weights.h5"
checkpoint = ModelCheckpoint(filepath, monitor=["acc"], verbose=1, mode='max')
callbacks_list = [checkpoint]

history = finetune_model.fit_generator(train_generator, epochs=NUM_EPOCHS, callbacks=callbacks_list,workers=6)


Epoch 1/30
11/11 [==============================] - 34s 3s/step - loss: 1.9918 - acc: 0.6920

Epoch 00001: saving model to checkpoints\VGG16sleeve_model_weights.h5
Epoch 2/30
11/11 [==============================] - 32s 3s/step - loss: 2.0550 - acc: 0.6930

Epoch 00002: saving model to checkpoints\VGG16sleeve_model_weights.h5
Epoch 3/30
11/11 [==============================] - 32s 3s/step - loss: 0.9133 - acc: 0.8185

Epoch 00003: saving model to checkpoints\VGG16sleeve_model_weights.h5
Epoch 4/30
11/11 [==============================] - 32s 3s/step - loss: 0.3372 - acc: 0.8859

Epoch 00004: saving model to checkpoints\VGG16sleeve_model_weights.h5
Epoch 5/30
11/11 [==============================] - 32s 3s/step - loss: 0.1425 - acc: 0.9544

Epoch 00005: saving model to checkpoints\VGG16sleeve_model_weights.h5
Epoch 6/30
11/11 [==============================] - 30s 3s/step - loss: 0.3198 - acc: 0.9201

Epoch 00006: saving model to checkpoints\VGG16sleeve_model_weights.h5
Epoch 7/30
11/11